# TODO: Image Classification Using PyTorch

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [ ]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [ ]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os
from sagemaker import get_execution_role
from sagemaker import Session

role = get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
print(f"Region: {region}")
bucket = sagemaker_session.default_bucket()
print(f"Current S3 Bucket: {bucket}")

In [ ]:
# Additional required imports

from sagemaker.tuner import(
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner
)

from sagemaker.Pytorch import Pytorch
from sagemaker.debugger import (
    Rule, ProfilerRule, rule_configs,
    DebuggerHookConfig, ProfilerConfig, FrameworkProfile
)

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

In [ ]:
local_dir = 'DogImages'
print('Uploading Dog Images..')
prefix = 'imageClassificationProject'
inputs = sagemaker_session.upload_data(path=local_dir, bucket=bucket, key_prefix=prefix)
print(f"Path ( S3 path ): {inputs}")

In [ ]:
inputs = path to Images
print(f"Path ( S3 path ): {inputs}")

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "batch-size": CategoricalParameter([64, 128]),
    "epochs": IntegerParameter(5, 10),
    "lr": ContinuousParameter(0.001, 0.1),
    "momentum": ContinuousParameter(0.7, 0.999),
}

In [ ]:
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test Loss: ([+-]?[0-9\\.]+)"}]


In [ ]:
# estimator

estimator = Pytorch(
    entry_point="hpo.py",
    role=role,
    py_version="py36",
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.g4dn.xlarge"
)
# tuner

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
    early_stopping_type="Auto"

)

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit({"training": inputs}, wait=True)

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
best_hyperparameters = {
    "batch_size": int(best_estimator.hyperparameters()["batch_size"].replace('"',"")),
    "epochs": int(best_estimator.hyperparameters()["epochs"],
    "lr": float(best_estimator.hyperparameters()["lr"],
    "momentum": float(best_estimator.hyperparameters()["momentum"],
    
}

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(num_steps=10)
)

debugger_config = DebuggerHookConfig(
    hook_parameters = {"train.save_interval": "100", "eval.save_interval": "10"}
)


In [ ]:
# TODO: Create and fit an estimator

estimator = Pytorch(
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    hyperparameters=best_hyperparameters,
    rules=rules,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_hook_config,
)

In [ ]:
estimator.fit({"training": inputs}. wait=True)

In [ ]:
# Plot a debugging output.

train_job_name = estimator.latest_training_job.name
print(f"Training Job Name: {train_job_name}")
print(f"Region Used: {region}")

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
trainingjob = TrainingJob(train_job_name, region)
trainingjob.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trials = create_trial(estimator.latest_job_debugger_artifacts_path())

In [ ]:
trials.tensor_names()

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"The location of the profiler report is {rule_output_path}")

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# Display the profiler output

! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
# autogenerated profiler report location

profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
]

In [ ]:
import iPython

iPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

In [ ]:
import shutil

shutil.make_archive("./profiler_report", "zip", "ProfilerReport")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()